In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml
import copy

import random
from itertools import chain, combinations, tee
import time

import os


# Functions

In [2]:
str_1 = "027___403206_Redwood_City___off"
"Redwood_City" in str_1

True

In [3]:
def station_to_city(station_name):
    if "East_Palo_Alto" in station_name:
        city_name = "East Palo Alto"
    elif "Palo_Alto" in station_name:
        city_name = "Palo Alto"
    elif "Menlo_Park" in station_name:
        city_name = "Menlo Park"
    elif "Redwood_City" in station_name:
        city_name = "Redwood City"
    elif "Belmont" in station_name:
        city_name = "Belmont"
    elif "San_Mateo" in station_name:
        city_name = "San Mateo"
    elif "Burlingame" in station_name:
        city_name = "Burlingame"
    elif "Millbrae" in station_name:
        city_name = "Millbrae"
    else:
        assert 1 == 0, "There should be no other case."
    return city_name

def income_range_avg(income_range_str):
    
    if income_range_str == "< $10,000":
        mean_income = 10000/2

    elif income_range_str == "$10,000 to $14,999":
        mean_income = (10000 + 15000)/2

    elif income_range_str == "$15,000 to $24,999":
        mean_income = (15000 + 25000)/2

    elif income_range_str == "$25,000 to $34,999":
        mean_income = (25000 + 35000)/2
        
    elif income_range_str == "$35,000 to $49,999":
        mean_income = (35000 + 50000)/2
        
    elif income_range_str == "$50,000 to $74,999":
        mean_income = (50000 + 75000)/2
        
    elif income_range_str == "$75,000 to $99,999":
        mean_income = (75000 + 100000)/2

    elif income_range_str == "$100,000 to $149,999":
        mean_income = (100000 + 150000)/2
    
    elif income_range_str == "$150,000 to $199,999":
        mean_income = (150000 + 200000)/2
    
    else:
        assert 1 == 0, "This case should not occur."
    
    return mean_income

# Import O-D Flow Data + Income Data:

In [4]:
directory_path = '../data/pems_flow___101_N_Sep_to_Nov_2024/'
df_od_flow_data = pd.read_csv(directory_path + 'data_od.csv')

dict_df_od_flow_data = {}

for column_name_full in list(df_od_flow_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_od_flow_data[column_name_full].tolist()
    else:
        dict_df_od_flow_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start Index" or category == "End Index":
                dict_df_od_flow_data[int(column_name_full)][category] \
                    = int(df_od_flow_data[column_name_full].tolist()[category_index])
            elif category == "O-D Flow (Initialization)" or category == "O-D Flow (Max Entropy)":
                dict_df_od_flow_data[int(column_name_full)][category] \
                    = float(df_od_flow_data[column_name_full].tolist()[category_index])
            else:
                dict_df_od_flow_data[int(column_name_full)][category] \
                    = df_od_flow_data[column_name_full].tolist()[category_index]
        
        dict_df_od_flow_data[int(column_name_full)]["Start City"] \
            = station_to_city(dict_df_od_flow_data[int(column_name_full)]["Start Station"])
        dict_df_od_flow_data[int(column_name_full)]["End City"] \
            = station_to_city(dict_df_od_flow_data[int(column_name_full)]["End Station"])
        
dict_df_od_flow_data
# dict_df_od_flow_data[0]["O-D Flow (Initialization)"]


{0: {'Start Index': 0,
  'End Index': 1,
  'Start Station': '001___402376_Palo_Alto___main',
  'End Station': '004___425696_Palo_Alto___off',
  'O-D Flow (Initialization)': 612.5396169,
  'O-D Flow (Max Entropy)': 612.5396169,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto'},
 1: {'Start Index': 0,
  'End Index': 5,
  'Start Station': '001___402376_Palo_Alto___main',
  'End Station': '010___403059_East_Palo_Alto___off',
  'O-D Flow (Initialization)': 98.0284297,
  'O-D Flow (Max Entropy)': 80.1904526,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto'},
 2: {'Start Index': 0,
  'End Index': 12,
  'Start Station': '001___402376_Palo_Alto___main',
  'End Station': '027___403206_Redwood_City___off',
  'O-D Flow (Initialization)': 309.2110456,
  'O-D Flow (Max Entropy)': 413.6495858,
  'Start City': 'Palo Alto',
  'End City': 'Redwood City'},
 3: {'Start Index': 0,
  'End Index': 14,
  'Start Station': '001___402376_Palo_Alto___main',
  'End Station': '032___410094_Redwood_

In [5]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_income_data = pd.read_csv(directory_path + 'data_income_raw___year_2023.csv')

dict_df_income_data = {}

for column_name_full in list(df_income_data.columns):
    if column_name_full == "Income":
        categories_list = df_income_data[column_name_full].tolist()
    else:
        dict_df_income_data[column_name_full] = {}
        for category_index, category in enumerate(categories_list):
            dict_df_income_data[column_name_full][category] \
                = float(df_income_data[column_name_full].tolist()[category_index])
                
dict_df_income_data


{'Palo Alto': {'< $10,000': 4.7,
  '$10,000 to $14,999': 3.1,
  '$15,000 to $24,999': 2.0,
  '$25,000 to $34,999': 2.5,
  '$35,000 to $49,999': 3.5,
  '$50,000 to $74,999': 7.8,
  '$75,000 to $99,999': 7.5,
  '$100,000 to $149,999': 13.2,
  '$150,000 to $199,999': 8.8,
  '>= $200,000': 46.9,
  'Median': 184068.0,
  'Mean': 264292.0},
 'East Palo Alto': {'< $10,000': 3.6,
  '$10,000 to $14,999': 2.3,
  '$15,000 to $24,999': 3.9,
  '$25,000 to $34,999': 4.9,
  '$35,000 to $49,999': 8.8,
  '$50,000 to $74,999': 14.9,
  '$75,000 to $99,999': 9.5,
  '$100,000 to $149,999': 19.3,
  '$150,000 to $199,999': 9.7,
  '>= $200,000': 23.1,
  'Median': 104832.0,
  'Mean': 138379.0},
 'Menlo Park': {'< $10,000': 1.9,
  '$10,000 to $14,999': 0.3,
  '$15,000 to $24,999': 2.2,
  '$25,000 to $34,999': 2.7,
  '$35,000 to $49,999': 5.0,
  '$50,000 to $74,999': 6.2,
  '$75,000 to $99,999': 5.6,
  '$100,000 to $149,999': 13.4,
  '$150,000 to $199,999': 9.0,
  '>= $200,000': 53.6,
  'Median': 206588.0,
  'Mea

# Mean Incomes, VoTs, Population Percentages

## Case 1: (2 eligible groups, 3 ineligible groups)

In [6]:
cities_list = list(dict_df_income_data.keys())

dict_income_groups_1 = {}
income_range_list_1 = ["< $15,000", "$15,000 to $35,000", "$35,000 to $100,000", \
                       "$100,000 to $200,000", ">= $200,000"]

for city_name in cities_list:
    dict_income_groups_1[city_name] = {}
    
    for income_range in income_range_list_1:
        
        if income_range == "< $15,000":
            percent_of_population = dict_df_income_data[city_name]["< $10,000"] \
                + dict_df_income_data[city_name]["$10,000 to $14,999"]
            mean_income = (income_range_avg("< $10,000") * dict_df_income_data[city_name]["< $10,000"] \
                + income_range_avg("$10,000 to $14,999") * dict_df_income_data[city_name]["$10,000 to $14,999"]) \
                / percent_of_population
            dict_income_groups_1[city_name][income_range] = [mean_income, percent_of_population]
            
        elif income_range == "$15,000 to $35,000":
            percent_of_population = dict_df_income_data[city_name]["$15,000 to $24,999"] \
                + dict_df_income_data[city_name]["$25,000 to $34,999"]
            mean_income = (income_range_avg("$15,000 to $24,999") * dict_df_income_data[city_name]["$15,000 to $24,999"] \
                + income_range_avg("$25,000 to $34,999") * dict_df_income_data[city_name]["$25,000 to $34,999"]) \
                / percent_of_population
            dict_income_groups_1[city_name][income_range] = [mean_income, percent_of_population]
            
        elif income_range == "$35,000 to $100,000":
            percent_of_population = dict_df_income_data[city_name]["$35,000 to $49,999"] \
                + dict_df_income_data[city_name]["$50,000 to $74,999"] \
                + dict_df_income_data[city_name]["$75,000 to $99,999"]
            mean_income = (income_range_avg("$35,000 to $49,999") * dict_df_income_data[city_name]["$35,000 to $49,999"] \
                + income_range_avg("$50,000 to $74,999") * dict_df_income_data[city_name]["$50,000 to $74,999"] \
                + income_range_avg("$75,000 to $99,999") * dict_df_income_data[city_name]["$75,000 to $99,999"]) \
                / percent_of_population
            dict_income_groups_1[city_name][income_range] = [mean_income, percent_of_population]
            
        elif income_range == "$100,000 to $200,000":
            percent_of_population = dict_df_income_data[city_name]["$100,000 to $149,999"] \
                + dict_df_income_data[city_name]["$150,000 to $199,999"]
            mean_income = (income_range_avg("$100,000 to $149,999") * dict_df_income_data[city_name]["$100,000 to $149,999"] \
                + income_range_avg("$150,000 to $199,999") * dict_df_income_data[city_name]["$150,000 to $199,999"]) \
                / percent_of_population
            dict_income_groups_1[city_name][income_range] = [mean_income, percent_of_population]
            
        elif income_range == ">= $200,000":
            percent_of_population = dict_df_income_data[city_name][">= $200,000"]          
            dict_income_groups_1[city_name][income_range] = [0, percent_of_population, 0]
        
        else:
            assert 1 == 0, "This case should not occur."
    
    if income_range == ">= $200,000":
        sum_of_income_mean_percent_products \
            = sum([dict_income_groups_1[city_name][income_range][0] * dict_income_groups_1[city_name][income_range][1] \
                   for income_range in income_range_list_1 if income_range != ">= 200,000"])
        mean_income = (dict_df_income_data[city_name]["Mean"] * 100 - sum_of_income_mean_percent_products) \
            / dict_income_groups_1[city_name][">= $200,000"][1]
        dict_income_groups_1[city_name][income_range][0] = mean_income

dict_income_groups_1


{'Palo Alto': {'< $15,000': [7980.7692307692305, 7.800000000000001],
  '$15,000 to $35,000': [25555.555555555555, 4.5],
  '$35,000 to $100,000': [68750.0, 18.8],
  '$100,000 to $200,000': [145000.0, 22.0],
  '>= $200,000': [464167.3773987207, 46.9, 0]},
 'East Palo Alto': {'< $15,000': [7923.728813559322, 5.9],
  '$15,000 to $35,000': [25568.181818181816, 8.8],
  '$35,000 to $100,000': [64352.409638554214, 33.2],
  '$100,000 to $200,000': [141724.1379310345, 29.0],
  '>= $200,000': [316867.96536796534, 23.1, 0]},
 'Menlo Park': {'< $15,000': [6022.727272727274, 2.1999999999999997],
  '$15,000 to $35,000': [25510.20408163265, 4.9],
  '$35,000 to $100,000': [64880.95238095239, 16.799999999999997],
  '$100,000 to $200,000': [145089.2857142857, 22.4],
  '>= $200,000': [549687.5, 53.6, 0]},
 'Redwood City': {'< $15,000': [7307.692307692308, 5.2],
  '$15,000 to $35,000': [26984.126984126982, 6.300000000000001],
  '$35,000 to $100,000': [65117.92452830189, 21.2],
  '$100,000 to $200,000': [14

In [7]:
dict_mean_income_estimate_1 = {}
dict_population_percentage_1 = {}
dict_VoT_1 = {}

dict_mean_income_estimate_1["Income Range"] = income_range_list_1
dict_population_percentage_1["Income Range"] = income_range_list_1
dict_VoT_1["Income Range"] = income_range_list_1

for city_name in cities_list:
    dict_mean_income_estimate_1[city_name] = []
    dict_population_percentage_1[city_name] = []
    dict_VoT_1[city_name] = []
    
    for income_range in income_range_list_1:
        dict_mean_income_estimate_1[city_name] += [dict_income_groups_1[city_name][income_range][0]]
        dict_population_percentage_1[city_name] += [dict_income_groups_1[city_name][income_range][1]]
        dict_VoT_1[city_name] += [dict_income_groups_1[city_name][income_range][0] / (40*52*60*2)]

In [9]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_mean_income_estimate_1 = pd.DataFrame(dict_mean_income_estimate_1)
df_mean_income_estimate_1.to_csv(directory_path + 'data_mean_income_estimate_1.csv', index = False)

df_population_percentage_1 = pd.DataFrame(dict_population_percentage_1)
df_population_percentage_1.to_csv(directory_path + 'data_population_percentage_1.csv', index = False)

df_VoT_1 = pd.DataFrame(dict_VoT_1)
df_VoT_1.to_csv(directory_path + 'data_VoT_1.csv', index = False)



## Case 2: (1 eligible group, 2 ineligible groups)

In [ ]:
cities_list = list(dict_df_income_data.keys())

dict_income_groups_2 = {}
income_range_list_2 = ["< $35,000", "$35,000 to $200,000", ">= $200,000"]

for city_name in cities_list:
    dict_income_groups_2[city_name] = {}
    
    for income_range in income_range_list_2:
        dict_income_groups_2[city_name][income_range] = np.zeros(2)
        
        if income_range == "< $35,000":
            percent_of_population = dict_df_income_data[city_name]["< $10,000"] \
                + dict_df_income_data[city_name]["$10,000 to $14,999"] \
                + dict_df_income_data[city_name]["$15,000 to $24,999"] \
                + dict_df_income_data[city_name]["$25,000 to $34,999"]
            mean_income = (income_range_avg("< $10,000") * dict_df_income_data[city_name]["< $10,000"] \
                + income_range_avg("$10,000 to $14,999") * dict_df_income_data[city_name]["$10,000 to $14,999"] \
                + income_range_avg("$15,000 to $24,999") * dict_df_income_data[city_name]["$15,000 to $24,999"] \
                + income_range_avg("$25,000 to $34,999") * dict_df_income_data[city_name]["$25,000 to $34,999"]) \
                / percent_of_population
            dict_income_groups_2[city_name][income_range] = [mean_income, percent_of_population]
            
        elif income_range == "$35,000 to $200,000":
            percent_of_population = dict_df_income_data[city_name]["$35,000 to $49,999"] \
                + dict_df_income_data[city_name]["$50,000 to $74,999"] \
                + dict_df_income_data[city_name]["$75,000 to $99,999"] \
                + dict_df_income_data[city_name]["$100,000 to $149,999"] \
                + dict_df_income_data[city_name]["$150,000 to $199,999"]
            mean_income = (income_range_avg("$35,000 to $49,999") * dict_df_income_data[city_name]["$35,000 to $49,999"] \
                + income_range_avg("$50,000 to $74,999") * dict_df_income_data[city_name]["$50,000 to $74,999"] \
                + income_range_avg("$75,000 to $99,999") * dict_df_income_data[city_name]["$75,000 to $99,999"] \
                + income_range_avg("$100,000 to $149,999") * dict_df_income_data[city_name]["$100,000 to $149,999"] \
                + income_range_avg("$150,000 to $199,999") * dict_df_income_data[city_name]["$150,000 to $199,999"]) \
                / percent_of_population
            dict_income_groups_2[city_name][income_range] = [mean_income, percent_of_population]
            
        elif income_range == ">= $200,000":
            percent_of_population = dict_df_income_data[city_name][">= $200,000"]          
            dict_income_groups_2[city_name][income_range] = [0, percent_of_population]
        
        else:
            assert 1 == 0, "This case should not occur."
    
    if income_range == ">= $200,000":
        sum_of_income_mean_percent_products \
            = sum([dict_income_groups_2[city_name][income_range][0] * dict_income_groups_2[city_name][income_range][1] \
                   for income_range in income_range_list_2 if income_range != ">= 200,000"])
        mean_income = (dict_df_income_data[city_name]["Mean"] * 100 - sum_of_income_mean_percent_products) \
            / dict_income_groups_2[city_name][">= $200,000"][1]
        dict_income_groups_2[city_name][income_range][0] = mean_income

dict_income_groups_2


In [ ]:
dict_mean_income_estimate_2 = {}
dict_population_percentage_2 = {}
dict_VoT_2 = {}

dict_mean_income_estimate_2["Income Range"] = income_range_list_2
dict_population_percentage_2["Income Range"] = income_range_list_2
dict_VoT_2["Income Range"] = income_range_list_2

for city_name in cities_list:
    dict_mean_income_estimate_2[city_name] = []
    dict_population_percentage_2[city_name] = []
    dict_VoT_2[city_name] = []
    
    for income_range in income_range_list_2:
        dict_mean_income_estimate_2[city_name] += [dict_income_groups_2[city_name][income_range][0]]
        dict_population_percentage_2[city_name] += [dict_income_groups_2[city_name][income_range][1]]
        dict_VoT_2[city_name] += [dict_income_groups_2[city_name][income_range][0] / (40*52*60*2)]


In [ ]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_mean_income_estimate_2 = pd.DataFrame(dict_mean_income_estimate_2)
df_mean_income_estimate_2.to_csv(directory_path + 'data_mean_income_estimate_2.csv', index = False)

df_population_percentage_2 = pd.DataFrame(dict_population_percentage_2)
df_population_percentage_2.to_csv(directory_path + 'data_population_percentage_2.csv', index = False)

df_VoT_2 = pd.DataFrame(dict_VoT_2)
df_VoT_2.to_csv(directory_path + 'data_VoT_2.csv', index = False)


# Demands

## Case 1: (2 eligible groups, 3 ineligible groups)

In [ ]:
# dict_df_od_flow_data

In [ ]:
dict_df_od_flow_data_augmented_1 = copy.deepcopy(dict_df_od_flow_data)

In [ ]:
for od_index in list(dict_df_od_flow_data.keys()):
    start_city_name = dict_df_od_flow_data[od_index]["Start City"]
    demand_total_od = dict_df_od_flow_data[od_index]["O-D Flow (Max Entropy)"]
    
    dict_df_od_flow_data_augmented_1[od_index]["Demand (eligible group, 1)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][0] / 100
    dict_df_od_flow_data_augmented_1[od_index]["VoT (eligible group, 1)"] = dict_VoT_1[start_city_name][0]
    dict_df_od_flow_data_augmented_1[od_index]["Demand (eligible group, 2)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][1] / 100
    dict_df_od_flow_data_augmented_1[od_index]["VoT (eligible group, 2)"] = dict_VoT_1[start_city_name][1]
    
    dict_df_od_flow_data_augmented_1[od_index]["Demand (ineligible group, 1)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][2] / 100
    dict_df_od_flow_data_augmented_1[od_index]["VoT (ineligible group, 1)"] = dict_VoT_1[start_city_name][2]
    dict_df_od_flow_data_augmented_1[od_index]["Demand (ineligible group, 2)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][3] / 100
    dict_df_od_flow_data_augmented_1[od_index]["VoT (ineligible group, 2)"] = dict_VoT_1[start_city_name][3]
    dict_df_od_flow_data_augmented_1[od_index]["Demand (ineligible group, 3)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][4] / 100
    dict_df_od_flow_data_augmented_1[od_index]["VoT (ineligible group, 3)"] = dict_VoT_1[start_city_name][4]


dict_df_od_flow_data_augmented_1


In [ ]:
# dict_df_od_flow_data

In [ ]:
# dict_df_od_flow_data

In [ ]:
dict_od_data_to_save_1 = {}
dict_od_data_to_save_1["Data Category"] \
    = ["Start Index", "End Index", "Start Station", "End Station", "Start City", "End City", \
       "O-D Flow (Initialization)", "O-D Flow (Max Entropy)", \
       "Demand (eligible group, 1)", "VoT (eligible group, 1)", \
       "Demand (eligible group, 2)", "VoT (eligible group, 2)", \
       "Demand (ineligible group, 1)", "VoT (ineligible group, 1)", \
       "Demand (ineligible group, 2)", "VoT (ineligible group, 2)", \
       "Demand (ineligible group, 3)", "VoT (ineligible group, 3)"]

for od_index in list(dict_df_od_flow_data.keys()):
    dict_od_data_to_save_1[od_index] = []
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data[od_index]["Start Index"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data[od_index]["End Index"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data[od_index]["Start Station"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data[od_index]["End Station"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data[od_index]["Start City"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data[od_index]["End City"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data[od_index]["O-D Flow (Initialization)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data[od_index]["O-D Flow (Max Entropy)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["Demand (eligible group, 1)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["VoT (eligible group, 1)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["Demand (eligible group, 2)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["VoT (eligible group, 2)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["Demand (ineligible group, 1)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["VoT (ineligible group, 1)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["Demand (ineligible group, 2)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["VoT (ineligible group, 2)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["Demand (ineligible group, 3)"]]
    dict_od_data_to_save_1[od_index] += [dict_df_od_flow_data_augmented_1[od_index]["VoT (ineligible group, 3)"]]

    

In [ ]:

# dict_od_data_to_save_1

directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_od_data_to_save_1 = pd.DataFrame(dict_od_data_to_save_1)
df_od_data_to_save_1.to_csv(directory_path + 'data_od_VoTs_demands_1.csv', index = False)


## Case 2: (1 eligible group, 2 ineligible groups)

In [ ]:
# dict_df_od_flow_data

In [ ]:
dict_df_od_flow_data_augmented_2 = copy.deepcopy(dict_df_od_flow_data)

In [ ]:
for od_index in list(dict_df_od_flow_data.keys()):
    start_city_name = dict_df_od_flow_data[od_index]["Start City"]
    demand_total_od = dict_df_od_flow_data[od_index]["O-D Flow (Max Entropy)"]
    
    dict_df_od_flow_data_augmented_2[od_index]["Demand (eligible group, 1)"] \
        = demand_total_od * dict_population_percentage_2[start_city_name][0] / 100
    dict_df_od_flow_data_augmented_2[od_index]["VoT (eligible group, 1)"] = dict_VoT_2[start_city_name][0]
    
    dict_df_od_flow_data_augmented_2[od_index]["Demand (ineligible group, 1)"] \
        = demand_total_od * dict_population_percentage_2[start_city_name][1] / 100
    dict_df_od_flow_data_augmented_2[od_index]["VoT (ineligible group, 1)"] = dict_VoT_2[start_city_name][1]
    dict_df_od_flow_data_augmented_2[od_index]["Demand (ineligible group, 2)"] \
        = demand_total_od * dict_population_percentage_2[start_city_name][2] / 100
    dict_df_od_flow_data_augmented_2[od_index]["VoT (ineligible group, 2)"] = dict_VoT_2[start_city_name][2]

dict_df_od_flow_data_augmented_2


In [ ]:
dict_df_od_flow_data

In [ ]:
dict_od_data_to_save_2 = {}
dict_od_data_to_save_2["Data Category"] \
    = ["Start Index", "End Index", "Start Station", "End Station", "Start City", "End City", \
       "O-D Flow (Initialization)", "O-D Flow (Max Entropy)", \
       "Demand (eligible group, 1)", "VoT (eligible group, 1)", \
       "Demand (ineligible group, 1)", "VoT (ineligible group, 1)", \
       "Demand (ineligible group, 2)", "VoT (ineligible group, 2)"]

for od_index in list(dict_df_od_flow_data.keys()):
    dict_od_data_to_save_2[od_index] = []
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data[od_index]["Start Index"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data[od_index]["End Index"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data[od_index]["Start Station"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data[od_index]["End Station"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data[od_index]["Start City"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data[od_index]["End City"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data[od_index]["O-D Flow (Initialization)"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data[od_index]["O-D Flow (Max Entropy)"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data_augmented_2[od_index]["Demand (eligible group, 1)"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data_augmented_2[od_index]["VoT (eligible group, 1)"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data_augmented_2[od_index]["Demand (ineligible group, 1)"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data_augmented_2[od_index]["VoT (ineligible group, 1)"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data_augmented_2[od_index]["Demand (ineligible group, 2)"]]
    dict_od_data_to_save_2[od_index] += [dict_df_od_flow_data_augmented_2[od_index]["VoT (ineligible group, 2)"]]
    

In [ ]:

# dict_od_data_to_save_2

directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_od_data_to_save_2 = pd.DataFrame(dict_od_data_to_save_2)
df_od_data_to_save_2.to_csv(directory_path + 'data_od_VoTs_demands_2.csv', index = False)


# Demands (One Edge Per City)

## Case 1: (2 eligible groups, 3 ineligible groups)

In [ ]:
# dict_df_od_flow_data

In [ ]:
od_cities_list = []
for key, od_info in dict_df_od_flow_data.items():
    if [od_info["Start City"], od_info["End City"]] not in od_cities_list:
        od_cities_list += [[od_info["Start City"], od_info["End City"]]]

od_flow_cities_max_entr = np.zeros(len(od_cities_list))
for key, od_info in dict_df_od_flow_data.items():
    od_cities_index = od_cities_list.index([od_info["Start City"], od_info["End City"]])
    od_flow_cities_max_entr[od_cities_index] += od_info["O-D Flow (Max Entropy)"]
    

print("od_cities_list:", od_cities_list)
print()
print("od_flow_cities_max_entr:", od_flow_cities_max_entr)



In [ ]:
dict_df_od_flow_data

In [ ]:
od_cities_dict = {}

for index, od_cities in enumerate(od_cities_list):
    od_cities_dict[index] = {}
    od_cities_dict[index]["Start City Index"] = cities_list.index(od_cities[0])
    od_cities_dict[index]["End City Index"] = cities_list.index(od_cities[1])
    od_cities_dict[index]["Start City"] = od_cities_list[index][0]
    od_cities_dict[index]["End City"] = od_cities_list[index][1]
    od_cities_dict[index]["O-D Flow (Max Entropy, Sum)"] = od_flow_cities_max_entr[index]

od_cities_dict

In [ ]:
od_cities_dict_augmented_1 = copy.deepcopy(od_cities_dict)

In [ ]:
for od_index in list(od_cities_dict.keys()):
#     od_cities_dict_augmented_1[]

    start_city_name = od_cities_dict[od_index]["Start City"]
    demand_total_od = od_cities_dict[od_index]["O-D Flow (Max Entropy, Sum)"]
    
    od_cities_dict_augmented_1[od_index]["Demand (eligible group, 1)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][0] / 100
    od_cities_dict_augmented_1[od_index]["VoT (eligible group, 1)"] = dict_VoT_1[start_city_name][0]
    od_cities_dict_augmented_1[od_index]["Demand (eligible group, 2)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][1] / 100
    od_cities_dict_augmented_1[od_index]["VoT (eligible group, 2)"] = dict_VoT_1[start_city_name][1]
    
    od_cities_dict_augmented_1[od_index]["Demand (ineligible group, 1)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][2] / 100
    od_cities_dict_augmented_1[od_index]["VoT (ineligible group, 1)"] = dict_VoT_1[start_city_name][2]
    od_cities_dict_augmented_1[od_index]["Demand (ineligible group, 2)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][3] / 100
    od_cities_dict_augmented_1[od_index]["VoT (ineligible group, 2)"] = dict_VoT_1[start_city_name][3]
    od_cities_dict_augmented_1[od_index]["Demand (ineligible group, 3)"] \
        = demand_total_od * dict_population_percentage_1[start_city_name][4] / 100
    od_cities_dict_augmented_1[od_index]["VoT (ineligible group, 3)"] = dict_VoT_1[start_city_name][4]

In [ ]:
dict_od_cities_data_to_save_1 = {}
dict_od_cities_data_to_save_1["Data Category"] \
    = ["Start City Index", "End City Index", "Start City", "End City", "O-D Flow (Max Entropy)", \
       "Demand (eligible group, 1)", "VoT (eligible group, 1)", \
       "Demand (eligible group, 2)", "VoT (eligible group, 2)", \
       "Demand (ineligible group, 1)", "VoT (ineligible group, 1)", \
       "Demand (ineligible group, 2)", "VoT (ineligible group, 2)", \
       "Demand (ineligible group, 3)", "VoT (ineligible group, 3)"]

for index, od_cities in enumerate(od_cities_list):
    dict_od_cities_data_to_save_1[index] = []
    dict_od_cities_data_to_save_1[index] += [od_cities_dict[index]["Start City Index"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict[index]["End City Index"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict[index]["Start City"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict[index]["End City"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict[index]["O-D Flow (Max Entropy, Sum)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["Demand (eligible group, 1)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["VoT (eligible group, 1)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["Demand (eligible group, 2)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["VoT (eligible group, 2)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["Demand (ineligible group, 1)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["VoT (ineligible group, 1)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["Demand (ineligible group, 2)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["VoT (ineligible group, 2)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["Demand (ineligible group, 3)"]]
    dict_od_cities_data_to_save_1[index] += [od_cities_dict_augmented_1[index]["VoT (ineligible group, 3)"]]


In [ ]:
# for value in dict_od_cities_data_to_save_1.values():
#     print(len(value))

In [ ]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_od_cities_data_to_save_1 = pd.DataFrame(dict_od_cities_data_to_save_1)
df_od_cities_data_to_save_1.to_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv', index = False)


## Case 2: (1 eligible group, 2 ineligible groups)

In [ ]:
od_cities_dict_augmented_2 = copy.deepcopy(od_cities_dict)

In [ ]:
for od_index in list(od_cities_dict.keys()):
#     od_cities_dict_augmented_2[]

    start_city_name = od_cities_dict[od_index]["Start City"]
    demand_total_od = od_cities_dict[od_index]["O-D Flow (Max Entropy, Sum)"]
    
    od_cities_dict_augmented_2[od_index]["Demand (eligible group, 1)"] \
        = demand_total_od * dict_population_percentage_2[start_city_name][0] / 100
    od_cities_dict_augmented_2[od_index]["VoT (eligible group, 1)"] = dict_VoT_2[start_city_name][0]
    
    od_cities_dict_augmented_2[od_index]["Demand (ineligible group, 1)"] \
        = demand_total_od * dict_population_percentage_2[start_city_name][1] / 100
    od_cities_dict_augmented_2[od_index]["VoT (ineligible group, 1)"] = dict_VoT_2[start_city_name][1]
    od_cities_dict_augmented_2[od_index]["Demand (ineligible group, 2)"] \
        = demand_total_od * dict_population_percentage_2[start_city_name][2] / 100
    od_cities_dict_augmented_2[od_index]["VoT (ineligible group, 2)"] = dict_VoT_2[start_city_name][2]

In [ ]:
dict_od_cities_data_to_save_2 = {}
dict_od_cities_data_to_save_2["Data Category"] \
    = ["Start City Index", "End City Index", "Start City", "End City", "O-D Flow (Max Entropy)", \
       "Demand (eligible group, 1)", "VoT (eligible group, 1)", \
       "Demand (ineligible group, 1)", "VoT (ineligible group, 1)", \
       "Demand (ineligible group, 2)", "VoT (ineligible group, 2)"]

for index, od_cities in enumerate(od_cities_list):
    dict_od_cities_data_to_save_2[index] = []
    dict_od_cities_data_to_save_2[index] += [od_cities_dict[index]["Start City Index"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict[index]["End City Index"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict[index]["Start City"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict[index]["End City"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict[index]["O-D Flow (Max Entropy, Sum)"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict_augmented_2[index]["Demand (eligible group, 1)"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict_augmented_2[index]["VoT (eligible group, 1)"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict_augmented_2[index]["Demand (ineligible group, 1)"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict_augmented_2[index]["VoT (ineligible group, 1)"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict_augmented_2[index]["Demand (ineligible group, 2)"]]
    dict_od_cities_data_to_save_2[index] += [od_cities_dict_augmented_2[index]["VoT (ineligible group, 2)"]]


In [ ]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_od_cities_data_to_save_2 = pd.DataFrame(dict_od_cities_data_to_save_2)
df_od_cities_data_to_save_2.to_csv(directory_path + 'data_cities_od_VoTs_demands_2.csv', index = False)


# Scratch Work

In [ ]:
ell = [1, 2, 3, 4, 5]
# ell[1:-1]
ell += [6]
ell